## Comparing alternative routes
In this notebook, we show how to setup a simple transport network and compare different routes to the same destination

We take the following steps:

1. [Imports](#1.-Imports)
2. [Create graph](#2.-Create-graph)
3. [Create vessel](#3.-Create-vessel)
4. [Run simulation(s)](#4.-Run-simulation(s))
5. [Inspect output](#5.-Inspect-output)

### 1. Imports
We start with importing required libraries

In [ ]:
# package(s) used for creating and geo-locating the graph
import networkx as nx  
import shapely.geometry
import pyproj

# package(s) related to the simulation (creating the vessel, running the simulation)
import datetime, time
import simpy
import opentnsim

# package(s) needed for inspecting the output
import pandas as pd

# package(s) needed for plotting
import matplotlib.pyplot as plt

print('This notebook is executed with OpenTNSim version {}'.format(opentnsim.__version__))

In [ ]:
# # package(s) related to time, space and id
# import datetime, time
# import platform

# # you need these dependencies (you can get these from anaconda)
# # package(s) related to the simulation
# import simpy

# # spatial libraries 
# import pyproj
# import shapely.geometry
# from simplekml import Kml, Style

# # package(s) for data handling
# import numpy as np
# import matplotlib.pyplot as plt

# # OpenTNSIM
# import opentnsim.core as core

# # Used for mathematical functions
# import math             

# # Used for making the graph to visualize our problem
# import networkx as nx  

### 2. Create graph
OpenTNSim works with mix-in classes to allow for flexibility in defining nodes. 

In [ ]:
# specify a number of coordinates along your route (coords are specified in world coordinates: lon, lat)
coords = [
    [0,0],
    [0,0.1], 
    [0.1,0.1], 
    [0.1,0]] 

In [ ]:
# make your preferred Site class out of available mix-ins.
Node = type('Site', (opentnsim.core.Identifiable, opentnsim.core.Locatable), {})

# create a list of nodes
nodes = []
for index, coord in enumerate(coords):
    data_node = {"name": str(index), "geometry": shapely.geometry.Point(coord[0], coord[1])}
    nodes.append(Node(**data_node))

In [ ]:
# create a graph based on coords and depths
FG = nx.DiGraph()

# add nodes
for node in nodes:
    FG.add_node(node.name, geometry = node.geometry)

# add edges
path = [
        [nodes[0], nodes[3]], # From node 0 to node 3 - so from node 0 to node 3 is one-way traffic   
        [nodes[0], nodes[1]], # From node 0 to node 1 - all other edges are two-way traffic
        [nodes[1], nodes[0]], # From node 1 to node 0
        [nodes[1], nodes[2]], # From node 1 to node 2
        [nodes[2], nodes[1]], # From node 2 to node 1
        [nodes[2], nodes[3]], # From node 2 to node 3
        [nodes[3], nodes[2]], # From node 3 to node 2
       ] 

for edge in path:
    FG.add_edge(edge[0].name, edge[1].name, weight = 1)

In [ ]:
# create a positions dict for the purpose of plotting
positions = {}
for node in FG.nodes:
    positions[node] = (FG.nodes[node]['geometry'].x, FG.nodes[node]['geometry'].y)

In [ ]:
# collect node labels
labels = {}
for node in FG.nodes:
    labels[node] = node

In [ ]:
# draw edges, nodes and labels.
nx.draw_networkx_edges(FG,  pos=positions, width=3, edge_color="red", alpha=1, arrowsize=20)
nx.draw_networkx_nodes(FG,  pos=positions, node_color="darkblue", node_size=600)
nx.draw_networkx_labels(FG, pos=positions, labels=labels, font_size=15, font_weight='bold', font_color="white")

plt.axis("off")
plt.show()

In [ ]:
# To show that moving from Node 4 to Node 1 is not possible
print("From 0 to 3:", nx.shortest_path_length(FG, "0", "3"))
print("From 3 to 0:", nx.shortest_path_length(FG, "3", "0"))

### 3. Create vessel
Vessel without graph, but with shortest path.

In [ ]:
# Make a class out of mix-ins
TransportResource = type('TransportResource', 
                         (opentnsim.core.Identifiable, 
                          opentnsim.core.Movable, 
                          opentnsim.core.Routeable), {})

data_vessel = {"env": None,
               "name": "Vessel 1",
               "route": None,
               "geometry": None,  # lon, lat
               "v": 1
              }

### 4. Run simulation(s)

In [ ]:
def calculate_distance(orig, dest):
    """method to calculate the greater circle distance in meters from WGS84 lon, lat coordinates"""
    
    wgs84 = pyproj.Geod(ellps='WGS84')
    distance = wgs84.inv(orig.x, orig.y, 
                         dest.x, dest.y)[2]
    
    return distance

In [ ]:
def calculate_distance_along_path(FG, path):
    """method to calculate the greater circle distance along path in meters from WGS84 lon, lat coordinates"""

    distance_path = 0

    for node in enumerate(path[:-1]):
        orig = nx.get_node_attributes(FG, "geometry")[path[node[0]]]
        dest = nx.get_node_attributes(FG, "geometry")[path[node[0]+1]]
        distance_path += calculate_distance(orig, dest)

        if node[0] + 2 == len(path):
                    break

    return distance_path

In [ ]:
def start(env, vessel):
    while True:
        vessel.log_entry("Start sailing", env.now, "", vessel.geometry)
        yield from vessel.move()
        vessel.log_entry("Stop sailing", env.now, "", vessel.geometry)
        
        if vessel.geometry == nx.get_node_attributes(FG, "geometry")[vessel.route[-1]]:
            break

In [ ]:
# first simulation is from Node 1 to Node 4
path_1 = nx.dijkstra_path(FG, "0", "3")

# second simulation is from Node 4 to Node 1
path_2 = nx.dijkstra_path(FG, "3", "0")

# collect paths in list
paths = [path_1, path_2]

In [ ]:
# run a simulation for each path in the list
for path in enumerate(paths):

    # Start simpy environment
    simulation_start = datetime.datetime.now()
    env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
    env.epoch = time.mktime(simulation_start.timetuple())
    
    # Add graph to environment
    env.FG = FG
    
    # create the transport processing resource
    vessel = TransportResource(**data_vessel)
    
    # Add environment and path to the vessel
    vessel.env = env
    vessel.route = path[1]
    vessel.geometry = FG.nodes[path[1][0]]['geometry']
    
    # Start the simulation
    env.process(start(env, vessel))
    env.run()
    
    df = pd.DataFrame.from_dict(vessel.log)
    display(df)

    print("Simulation of path {} took {:.1f} seconds".format(path[0] + 1, (env.now - simulation_start.timestamp())))  

    print("Distance of path {} is {:.1f} meters".format(path[0] + 1, calculate_distance_along_path(FG, path[1])))

### 5. Inspect output

In [ ]:
df = pd.DataFrame.from_dict(vessel.log)
df